# Praticando transformação geométrica 2D

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [21]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

### Inicializando janela

In [22]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(800, 800, "Transformação Geométrica", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4).
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação.

In [23]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat;
        void main(){
            gl_Position = mat * vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [24]:
fragment_code = """
        void main(){
            gl_FragColor = vec4(1.0,0.0,0.0,1.0);
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [25]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [26]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [27]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [28]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [29]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [30]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [31]:
# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(6, [("position", np.float32, 2)])

In [32]:
# preenchendo as coordenadas de cada vértice
vertices['position'] = [
                            ( 0.00, +0.05), 
                            (-0.05, -0.05), 
                            (+0.05, -0.05),
                            ( 0.00, +0.025), 
                            (-0.025, -0.025), 
                            (+0.025, -0.025)
                        ]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [33]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [34]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [35]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [36]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [37]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [38]:
flag_tiro = False

# translacao
x_inc = 0.0
y_inc = 0.0

# rotacao
r_inc = 0.0
c_x = 0.0
c_y = 0.0


# coeficiente de escala
s_inc = 1.0


def key_event(window,key,scancode,action,mods):
    global x_inc, y_inc, r_inc, s_inc, flag_tiro
    
    if key == 263: x_inc -= 0.03
    if key == 262: x_inc += 0.03

    if key == 265: y_inc += 0.03
    if key == 264: y_inc -= 0.03
        
    # if key == 65: r_inc += 0.1
    # if key == 83: r_inc -= 0.1
        
    if key == 81: s_inc += 0.1
    if key == 87: s_inc -= 0.1

    if key == 32: flag_tiro = True
        
    print(key)
        
    #print(key)
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    
glfw.set_mouse_button_callback(window,mouse_event)


def cursor_event(window, xpos, ypos):
    global c_x, c_y
    c_x = xpos
    c_y = ypos
    #print(f"Posição do cursor: ({xpos}, {ypos})")

# Associe a função de callback à janela
glfw.set_cursor_pos_callback(window, cursor_event)



### Nesse momento, nós exibimos a janela!


In [39]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays()

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

In [40]:
t_x = 0.0
t_y = 0.0
borda_y_cima = 0.96
borda_y_baixo = -0.95
borda_x_cima= 0.96
borda_x_baixo = -0.95
angulo = 0.0
s_x = 1.0
s_y = 1.0
angulo_tiro = 0.0
x_tiro = 0.0
y_tiro = 0.0


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c


while not glfw.window_should_close(window):

    t_x = x_inc
    t_y = y_inc

    if flag_tiro:
        y_tiro += 0.0005
    else:
        x_tiro = t_x
        y_tiro = t_y

    if t_y >= borda_y_cima:
        t_y = borda_y_cima
    elif t_y <= borda_y_baixo:
        t_y = borda_y_baixo

    if t_x >= borda_x_cima:
        t_x = borda_x_cima
    elif t_x <= borda_x_baixo:
        t_x = borda_x_baixo
    
    if y_tiro >= borda_y_cima:
        flag_tiro = False

    angulo = 30
    
    s_x = s_inc
    s_y = s_inc

    # Calcular a direção do objeto para o cursor
    direction = np.array([c_x, c_y]) - np.array([t_x, t_y])
    angulo = np.arctan2(direction[1], direction[0])

    #c = math.cos( math.radians(angulo) )
    c = np.cos(angulo)
    #s = math.sin( math.radians(angulo) )
    s = np.sin(angulo)
    
    glfw.poll_events() 

    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    
    #Draw Triangle
    
    mat_rotation = np.array([  c  , -s , 0.0, 0.0, 
                               s  , c  , 0.0, 0.0, 
                               0.0, 0.0, 1.0, 0.0, 
                               0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_scale =    np.array([  s_x  , 0.0 , 0.0, 0.0, 
                               0.0  , s_y  , 0.0, 0.0, 
                               0.0, 0.0, 1.0, 0.0, 
                               0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_translation = np.array([  1.0, 0.0, 0.0, t_x, 
                                  0.0, 1.0, 0.0, t_y, 
                                  0.0, 0.0, 1.0, 0.0, 
                                  0.0, 0.0, 0.0, 1.0], np.float32)
    
    
    #print(mat_rotation,"\n\n")

    mat_transform = multiplica_matriz(mat_scale, mat_translation)
    mat_transform = multiplica_matriz(mat_transform,mat_rotation)
        
    loc = glGetUniformLocation(program, "mat")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
    glDrawArrays(GL_TRIANGLES, 0, 3)

    mat_translation = np.array([  1.0, 0.0, 0.0, x_tiro, 
                                  0.0, 1.0, 0.0, y_tiro, 
                                  0.0, 0.0, 1.0, 0.0, 
                                  0.0, 0.0, 0.0, 1.0], np.float32)
    
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)
    glDrawArrays(GL_TRIANGLES, 4, 3)

    glfw.swap_buffers(window)

glfw.terminate()

32
32


32
32
263
263
264
264
264
264
264
264
264
264
264
264
264
264
262
264
262
262
262
262
262
262
262
262
32
262
32
